In [ ]:
#r "nuget: FSharp.Data"
// #r "nuget: Microsoft.Data.Analysis"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET.Interactive,  2.0.0-preview.8"

#load "UnixTime.fsx"
#load "AssetPrices.fsx"
#load "Graph.fsx"

open Plotly.NET
open UnixTime;
open AssetPrices
open Graph
open FSharp.Data
open FSharp.Stats

let show a = printf "%A\n" a

type PossiblyUndefined =
    | Yeah of decimal
    | Nah

type Vault = 
    { Time : int64
    ; Collateral : decimal
    ; Debt : decimal
    ; Price : decimal
    ; TargetRatio : decimal
    ; LowerRatio : decimal
    ; UpperRatio : decimal
    ; Rebalances : uint
    } with
    member this.CollateralValue = this.Collateral * this.Price
    member this.ExcessCollateralValue = this.CollateralValue - this.Debt
    member this.ExcessCollateral = (this.ExcessCollateralValue / this.Price)
    member this.Ratio = 
        match this.Debt with
        | 0M -> Nah
        | _ -> Yeah(this.CollateralValue / this.Debt)
    member this.TargetLeverage = 1M/(this.TargetRatio - 1M) + 1M
    member this.ShouldRebalance = 
        match this.Ratio with
        | Nah -> true
        | Yeah ratio -> ratio < this.LowerRatio || ratio > this.UpperRatio


let rebalance (vault:Vault) = 
    // todo : add fees (gas, exchange, defisaver, slippage)
    let newCollateralValue = vault.ExcessCollateralValue * vault.TargetLeverage
    let newCollateral = newCollateralValue / vault.Price
    let collateralValueDiff = newCollateralValue - vault.CollateralValue
    let newDebt = newCollateralValue * (vault.TargetLeverage - 1M) / vault.TargetLeverage
    { vault with 
          Collateral = max newCollateral 0M
        ; Debt = max newDebt 0M
        ; Rebalances = vault.Rebalances + 1u
    }

let nextVault vault (candle:Candle) = 
    let preRebalanceVault = { vault with Time = candle.Time; Price = candle.Close }
    if preRebalanceVault.ShouldRebalance then
        rebalance preRebalanceVault 
    else
        preRebalanceVault

let create collateral time price targetRatio tollerance = 
    let init = 
        { Time = time
        ; Collateral = collateral
        ; Debt = 2000.0M
        ; Price = price
        ; TargetRatio = targetRatio
        ; LowerRatio = targetRatio - tollerance
        ; UpperRatio = targetRatio + tollerance
        ; Rebalances = 0u
        }
    init

let startingCollateral = 100M
let targetRatio = 2M
let range = 0.3M
let startDate = (toEpochTime 2021 7 30)

// makes a list of vaults over time that can be anylized


let vaultList startingCollateral targetRatio range startDate endDate = 
    let update (vaults: Vault list) (candle: Candle)  =
        match vaults with
        | [] -> [create startingCollateral candle.Time candle.Close targetRatio range]
        | head::tail -> 
            // printf "%A" head
            [nextVault head candle] @ vaults

    ethPrices startDate endDate
    |> Array.fold update []

let vaults = vaultList 100M 2M 0.3M (toEpochTime 2021 7 30) now
let last l = List.head l
let first l = vaults.[vaults.Length - 1]
let actual = (List.head vaults).ExcessCollateral / vaults.[vaults.Length - 1].ExcessCollateral
let expected = (List.head vaults).Price / vaults.[vaults.Length - 1].Price
let rebalances = (List.head vaults).Rebalances
(actual, expected, actual / expected, rebalances)

let time = vaults |> List.map (fun x -> x.Time |> toDateTime)
let actualCollateral = vaults |> List.map (fun x -> x.ExcessCollateral)
let predictedCollateral = 
    let firstPrice = (first vaults).Price
    vaults |> List.map (fun x-> startingCollateral * x.Price / firstPrice) 

drawChart time [("Actual", actualCollateral);("Predicted", predictedCollateral)] 

// [
//     Chart.Line(time, actualCollateral)
//     |> Chart.withTraceName(Name="Actual")
//     |> Chart.withLineStyle(Width=2.0, Dash=StyleParam.DrawingStyle.Solid)

//     Chart.Line(time, predictedCollateral)
//     |> Chart.withTraceName(Name="Predicted")
//     |> Chart.withLineStyle(Width=2.0, Dash=StyleParam.DrawingStyle.Solid) 
// ]
// |> Chart.combine
// |> Chart.withXAxisStyle("Time")
// |> Chart.withYAxisStyle("Collateral")


Installed Packages FSharp.Data, 4.2.4 FSharp.Stats, 0.4.2 Plotly.NET.Interactive, 2.0.0-preview.8

<!-- Plotly chart will be drawn inside this DIV -->